In [ ]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.DEBUG)

In [ ]:
# Council imports

from council.contexts import AgentContext, Budget, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource
from filter import WritingAssistantFilter 

In [ ]:
# Read environment variables and initialize OpenAI LLM instance

import os
import dotenv

dotenv.load_dotenv()
openai_llm = OpenAILLM.from_env()
budget = float(os.getenv("BUDGET"))  # budget in seconds

In [ ]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [ ]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [ ]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    [outline_chain, writer_chain],
    top_k_execution_plan=5
)

In [ ]:
# Filter

filter = WritingAssistantFilter(
    openai_llm,
    controller.state
)

In [ ]:
# Initialize Agent

agent = Agent(controller, BasicEvaluatorWithSource(), filter)

In [ ]:
# Run Agent

chat_history = ChatHistory.from_user_message("Write a detailed research article about the history of box manufacturing.")
run_context = AgentContext.from_chat_history(chat_history, Budget(budget))
result = agent.execute(run_context)

In [ ]:
print(result.messages[-1].message.message)

In [ ]:
# If you get a TimeoutError or result.messages is empty, your budget is not enough to complete the article writing
# but you can still access generated outline and article directly:

print("Outline: ")
print(agent.controller.state.outline)
print("\n\n----------------------------------------------------------\n\n")
print("Article: ")
print(agent.controller.state.article)

In [ ]:
chat_history.add_user_message("Can you please add a section (including subsections) on boxes in popular culture?")
run_context = AgentContext.from_chat_history(chat_history, Budget(budget))
result = agent.execute(run_context)

In [ ]:
print(result.messages[-1].message.message)